
# Data operations for prepping the WorldBank_Debt Dataset 

### Summary: 
The dataset [`world_bank_international_debt`](https://pantheon.corp.google.com/marketplace/product/the-world-bank/international-debt?project=paulkamau) ccontains both national and regional debt statistics captured by over 200 economic indicators. Time series data is available for those indicators from 1970 to 2015 for reporting countries.

Multiple tables have been used to compile the data. The data was flattened to create a single working set: 

- Country series definitions
- Country summary
- International debt
- Series summary
- Series times

### Actions: 
1. Created a pivot table which takes the indicator codes as rows and tranforms them into columns in a new table. 
1. Created a merged table of the countries_data and the new pivot table we made
1. Created a table definition of the indicator_code and their description
1. Created the indicator_code description table dictionary

### 1. Pivot Query. Convert the row values into column values 

<!-- SQL to create a pivot table which reates column data out of the row data from internation_debt table -->

original table we want: 
  `bigquery-public-data.world_bank_intl_debt.international_debt`

New pivot table with everything: 
--paulkamau.WorldBank_Debt.international_debt_pivot

```sql

DECLARE IndicatorCodes STRING;

SET IndicatorCodes = (
  SELECT 
    CONCAT('("', STRING_AGG(DISTINCT replace(indicator_code,".","_"), '", "'), '")'),
  FROM `bigquery-public-data.world_bank_intl_debt.international_debt` where value is not null
);


EXECUTE IMMEDIATE format("""
CREATE TABLE IF NOT EXISTS `paulkamau.WorldBank_Debt.international_debt_pivot` as 
SELECT * FROM
(
  SELECT country_name, country_code, replace(indicator_code,".","_") as ic,value
  FROM `bigquery-public-data.world_bank_intl_debt.international_debt`
) 
PIVOT
(
  MAX(value)
  FOR ic in %s
)
ORDER BY country_name ASC
""",
IndicatorCodes);

```

New pivot table with everything: 
--paulkamau.WorldBank_Debt.international_avg_debt_pivot

```sql

DECLARE IndicatorCodes STRING;

SET IndicatorCodes = (
  SELECT 
    CONCAT('("', STRING_AGG(DISTINCT replace(indicator_code,".","_"), '", "'), '")'),
  FROM `bigquery-public-data.world_bank_intl_debt.international_debt` where value is not null
);


EXECUTE IMMEDIATE format("""
CREATE TABLE IF NOT EXISTS `paulkamau.WorldBank_Debt.international_avg_debt_pivot` as 
SELECT * FROM
(
  SELECT country_name, country_code, replace(indicator_code,".","_") as ic,value
  FROM `bigquery-public-data.world_bank_intl_debt.international_debt`
) 
PIVOT
(
  MAX(value)
  FOR ic in %s
)
ORDER BY country_name ASC
""",
IndicatorCodes);

```

## Create the country_data table 

```sql

CREATE TABLE IF NOT EXISTS `paulkamau.WorldBank_Debt.countries_data` as 
SELECT 
country_code, short_name, currency_unit, income_group, national_accounts_base_year, sna_price_valuation, system_of_national_accounts, system_of_trade, government_accounting_concept
 FROM `bigquery-public-data.world_bank_intl_debt.country_summary`
where system_of_trade is not null;

```


## Create the indicator_code_description table 

```sql

CREATE TABLE IF NOT EXISTS `paulkamau.WorldBank_Debt.indicators_code_desc` as 
SELECT distinct(series_code), topic, indicator_name, short_definition, unit_of_measure, periodicity
 FROM `bigquery-public-data.world_bank_intl_debt.series_summary`;

--  then create a new column and copy the series_code column data into the new column and replace the 'periods' with 'underscores'. This is because the columns are built with underscores 

ALTER TABLE `paulkamau.WorldBank_Debt.indicators_code_desc`
ADD COLUMN series_code_new STRING;


-- copy the data from one column into the other
UPDATE `paulkamau.WorldBank_Debt.indicators_code_desc`
SET series_code_new = series_code
where series_code is not null;

-- update the column data with underscores
UPDATE `paulkamau.WorldBank_Debt.indicators_code_desc`
SET series_code_new = REPLACE(series_code_new,'.','_')
where series_code = series_code_new;

```